# Tree-based models - CART

In this section we will discuss the use of CART (**C**lassification **A**nd **R**egression **T**rees) and more specifically, we will use regression trees to predict claim frequencies.

In [ ]:
require("arrow")
# require("CASdatasets") - Not necessary, as we saved the data as parquet file
require("rpart")
require("rpart.plot")
require("caret")
require("ggplot2")


options(repr.plot.width = 8, repr.plot.height = 6, repr.plot.res = 150)

In [ ]:
dataset = read_parquet(file = "../data/dataset.parquet")

When performing machine learning, one way to assess the predictability of the model is to use *fresh* data, not used in the *learning* process and compute some metric. This is also a good way to check whether the model is overfitting the data.

In order to have *fresh* data available at a later stage, it is common to split the dataset into two:
 - a training set. This data is used to estimate/learn the model.
 - a test set. This data is used to check the performance of the model, once it has been learned.
 
The package *caret* provides a lot of helpful functions. One of them is *createDataPartition*. Let us now split the data into two pieces (80% - 20%), by relying on stratified sampling. We use stratified sampling to obtain a similar distribution of the number of claims in boths the training and the test set. 

In [ ]:
set.seed(21)
in_training = createDataPartition(dataset$ClaimNb, times = 1, p = 0.8, list = FALSE)
training_set = dataset[in_training, ]
testing_set = dataset[-in_training, ]

Let us check the distribution of the variable *ClaimNb* among these two groups.

In [ ]:
temp = dataset
temp$training = 0
temp[in_training, "in_train_set"] = 1
temp[-in_training, "in_train_set"] = 0

summary_table = 100*round(prop.table(with(temp, table(in_train_set, ClaimNb)), 1), 5)
summary_table
rm(temp)

## Package CART

We will use the package CART which allows to compute regression trees. rpart can be used for regression and classification. It also implements a method for *Poisson* data.

### Quick Example

Let us start with a simple example, using two covariates:



In [ ]:
m0_rpart = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 method = "poisson", 
                 control = rpart.control(cp = 0.01))
summary(m0_rpart)

It appears that the tree has a single node and has not been split further. This comes from the complexity parameter which penalizes the splitting. **By default**, the complexity parameter **cp = 0.01**, which is often too large for Poisson data with low frequencies.

Let us put **cp = 0**, but to keep a small tree we will also impose a maximum depth of 3.

In [ ]:
m0_rpart = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0, maxdepth = 3))
summary(m0_rpart)

The easiest way to interpret a CART is probably to plot it (if it is not too large, though!). This can be achieved with the function *rpart.plot* from the package *rpart.plot*.

In [ ]:
rpart.plot(m0_rpart, 
           type = 5, 
           extra = 101, 
           under = FALSE, 
           fallen.leaves = TRUE,
           digits = 3)

If the tree is too large, we will probably have some overfitting. To prevent overfitting, we can play with the complexity parameter cp. A good approach is to compute the whole tree, without any penalty (i.e. complexity parameter is set to 0) and afterwards prune to tree.

In [ ]:
m0_rpart = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0))
rpart.plot(m0_rpart)

In [ ]:
rpart.plot(prune(m0_rpart, cp = 9e-04), 
           type = 5, 
           extra = 101, 
           under = FALSE, 
           fallen.leaves = TRUE,
           digits = 3)

We also see that in some terminal nodes (i.e. leaves), the number of observations (and of claims) is very low. We can set a minimum number of observation in any terminal node using minbucket

In [ ]:
m0_rpart = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0, maxdepth = 3, minbucket = 1000))
rpart.plot(m0_rpart)

## Complexity Parameter

Playing around with the complexity parameter will yield sub-trees of the fully developped tree (i.e. the one with cp = 0).

We check verify on one example that increasing the complexity parameter *cp* will only prune the tree differently and that by increasing *cp*, we will obtain a subtree. The common parts of the three trees below (i.e., the first split) are identical.

In [ ]:
tree1 = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0.0063))
rpart.plot(tree1,
           type = 5, 
           extra = 101, 
           under = FALSE, 
           fallen.leaves = TRUE,
           digits = 3)

tree2 = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0.0013))
rpart.plot(tree2,
           type = 5, 
           extra = 101, 
           under = FALSE, 
           fallen.leaves = TRUE,
           digits = 3)


tree3 = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(cp = 0.0011))
rpart.plot(tree3,
           type = 5, 
           extra = 101, 
           under = FALSE, 
           fallen.leaves = TRUE,
           digits = 3)

## Cross-validation

Let us now find the optimal tree, by using cross-validation. We will again only use the variable DriverAge and CarAge in this section. By default, rpart will perform 10-fold cross-validation, using the option xval = 10. (Remark: The whole process of how the cross-validation is operated in described in Section 4.2 of rpart’s vignette: https://cran.r-project.org/web/packages/rpart/vignettes/longintro.pdf).

Essentially (and with some shortcuts), we can summarize the method as follows:

1. Estimate the full tree (cp = 0), on the whole data. We obtain a list of complexity parameters $\alpha_i$. We compute the $\beta$s are the geometric mean of two successive $\alpha_i$s. To each $\beta_i$ corresponds a subtree of the full tree.
2. Split the whole data into *folds* (generally 10 folds).
    - Estimate the tree using all folds but one. 
    - Using the list of $\beta_i$s from the previous step, we prune the tree to differents subtrees.
    - Compute the loss function ("risk") of these subtrees on the left-out fold. For each $\beta_i$ we obtain a loss.
    - Loop on the folds. We obtain for each $\beta_i$ as many losses as there are folds. We aggregate these losses by summing.
    - After agregation, for each $\beta_i$ we have a single loss.
3. The optimal complexity parameter $cp^*$ is defined as the $\beta_i$ with the smallest loss (see remark below). The optimal tree is defined as the subtree with cp = $cp^*$

Remark: We may also use the 1-SE rule, which will yield the largest $\beta_i$ whose corresponding loss is in the 1-standard error window of the smallest loss.

In [ ]:
m0_rpart = rpart(cbind(Exposure, ClaimNb) ~ DriverAge + CarAge, 
                 data = training_set,
                 control = rpart.control(
                             cp = 3e-5, 
                             xval = 10))
printcp(m0_rpart)

In [ ]:
plotcp(m0_rpart)

In [ ]:
head(m0_rpart$cptable, 10)

Let us see the optimal tree.

In [ ]:
cp_star = m0_rpart$cptable[which.min(m0_rpart$cptable[, 4]), 1]
print(cp_star)

rpart.plot(prune(m0_rpart, cp = cp_star), type = 5, extra = 101, under = FALSE, fallen.leaves = FALSE,
    digits = 3)

Using the 1-SE error, we need to compute the greatest cp parameters that enters the 1-SE window

In [ ]:
SE = m0_rpart$cptable[which.min(m0_rpart$cptable[, 4]), 5]
min_error = m0_rpart$cptable[which.min(m0_rpart$cptable[, 4]), 4]
boundary = min_error + SE
boundary

# We need to find the first cp such that xerror < boundary.
row_optimal = which(m0_rpart$cptable[, 4] < boundary)[1]
cp_star = m0_rpart$cptable[row_optimal, 1]
print(cp_star)

rpart.plot(prune(m0_rpart, cp = cp_star), type = 5, extra = 101, under = FALSE, fallen.leaves = FALSE,
    digits = 3)

## Using all covariates

Let us now include all the covariates from the dataset

In [ ]:
m1_rpart = rpart(cbind(Exposure, ClaimNb) ~ Power + CarAge + DriverAge + Brand +
    Gas + Region + Density, data = training_set, method = "poisson", control = rpart.control(cp = 0,
    xval = 10, minbucket = 1000))
printcp(m1_rpart)

In [ ]:
plotcp(x = m1_rpart, minline = TRUE, col = "red")

In [ ]:
# Compute boundary for 1SE-rule
SE = m1_rpart$cptable[which.min(m1_rpart$cptable[, 4]), 5]
min_error = m1_rpart$cptable[which.min(m1_rpart$cptable[, 4]), 4]
boundary = min_error + SE

# Plot cross-validation error and 1SE_rule boundary.
ggplot() + geom_line(aes(x = m1_rpart$cptable[, 1], y = m1_rpart$cptable[, 4]))+
geom_point(aes(x = m1_rpart$cptable[, 1], y = m1_rpart$cptable[, 4]))+
geom_hline(aes(yintercept = boundary), color="red")+
scale_x_continuous(name = "CP") + scale_y_continuous(name = "xerror") + theme_bw()

If we take the value of cp that minimizes the error, we find

In [ ]:
cp_star = m1_rpart$cptable[which.min(m1_rpart$cptable[, 4]), 1]
cp_star

Let us plot the optimal tree

In [ ]:
m2_rpart = prune(m1_rpart, cp = cp_star)
rpart.plot(m2_rpart, type = 5, extra = 101, under = FALSE, fallen.leaves = TRUE,
    digits = 3, cex = 0.60)

There is a possibility to extract a variable importance metric.

In [ ]:
plotdata = data.frame(m2_rpart$variable.importance)
names(plotdata) = 'importance'
plotdata$var = rownames(plotdata)

ggplot(plotdata,aes(x =reorder(var,importance), y=importance)) + 
geom_bar(stat='identity')+coord_flip()+
scale_x_discrete(name="Variable") + 
theme_bw()

Finally, let us compute the deviance on the testing_set.

In [ ]:
deviance_poisson = function(x_obs, x_pred) {
    2 * (sum(dpois(x = x_obs, lambda = x_obs, log = TRUE)) - sum(dpois(x = x_obs,
        lambda = x_pred, log = TRUE)))
}

deviance_poisson(x_obs = testing_set$ClaimNb, x_pred = predict(m2_rpart, testing_set) *
    testing_set$Exposure)

If we compute the deviance on the full tree (not the pruned tree), we obtain

In [ ]:
deviance_poisson(x_obs = testing_set$ClaimNb, x_pred = predict(m1_rpart, testing_set) *
    testing_set$Exposure)

## Bagging of trees

Let us create the bootstrap samples.

In [ ]:
set.seed(85)
bootstrap_samples = createResample(training_set$ClaimNb, times = 50)

For each sample, we estimate a CART with the optimal complexity parameter found previously. Each tree, gives us an estimation of the claim frequency, which we average.

In [ ]:
bagg_cart = lapply(bootstrap_samples, function(X) {
    rpart(cbind(Exposure, ClaimNb) ~ Power + CarAge + DriverAge + Brand + Gas + Region + Density, 
          data = training_set[X, ], 
          method = "poisson", 
          control = rpart.control(cp = cp_star,
                                  xval = 0))
})

In [ ]:
pred = lapply(bagg_cart, function(X) {
    predict(X, testing_set) * testing_set$Exposure
})

pred = do.call(cbind, pred)
pred = apply(pred, 1, mean)

In [ ]:
deviance_poisson(x_obs = testing_set$ClaimNb, x_pred = pred)